In [33]:
import json
import pandas as pd

basepath = '/home/xilini/par-gen/01-par-gen/model/scores/'

with open('/home/xilini/par-gen/01-par-gen/data/dataset_paragraphs_v1.json', 'r') as s:
    data = json.load(s)['images']
with open(basepath + 'VISUAL-beam_2_cidertest.json', 'r') as f1:
    VISUAL = json.load(f1)
with open(basepath + 'LANGUAGE-beam_2_cidertest.json', 'r') as f2:
    LANGUAGE = json.load(f2)
with open(basepath + 'VISUAL_LANGUAGE-beam_2_cidertest.json', 'r') as f3:
    VIS_LANG = json.load(f3)

In [35]:
# choose randomly 50 images and generate json file with paths to these images

stanford_path = 'https://cs.stanford.edu/people/rak248/'
image_loc = {}
for item in data:
    if item['split'] == 'test':
        filepath = item['filepath']
        image_id = item['stanford_par_id']
        image_path = stanford_path + str(filepath) + '/' + str(image_id) + '.jpg'
        image_loc[image_id] = image_path

import random
images = list(image_loc.keys())
random.seed(4)
random.shuffle(images)

images = images[:50]

random_dict = {}
for k in images:
    if k in image_loc.keys():
        random_dict[k] = image_loc[k]

with open('/home/xilini/par-gen/01-par-gen/data/stanford_loc.json', 'w') as st:
    json.dump(random_dict, st)

In [37]:
def make_df(data_input, par_dict):
    csv_dict = {}
    csv_dict['Image'] = []
    csv_dict['Paragraph'] = []

    for item in data_input:
        if item['image_id'] in par_dict.keys():
            image_link = par_dict[item['image_id']]
            hyps = item['hypotheses']
            csv_dict['Image'].append(image_link)
            csv_dict['Paragraph'].append(hyps)
            
    return csv_dict

In [42]:
vis_df = make_df(VISUAL, random_dict)
vis_csv_df = pd.DataFrame(data=vis_df)

In [44]:
lang_df = make_df(LANGUAGE, random_dict)
lang_csv_df = pd.DataFrame(data=lang_df)

In [45]:
vis_lang_df = make_df(VIS_LANG, random_dict)
vis_lang_csv_df = pd.DataFrame(data=vis_lang_df)

In [61]:
vis_csv_df['Model'] = 'VISUAL'
lang_csv_df['Model'] = 'LANGUAGE'
vis_lang_csv_df['Model'] = 'VIS_LANG'

In [62]:
vis_csv_df.shape

(50, 3)

In [63]:
lang_csv_df.shape

(50, 3)

In [64]:
vis_lang_csv_df.shape

(50, 3)

In [65]:
final_concat = pd.concat([vis_csv_df, lang_csv_df, vis_lang_csv_df])

In [66]:
final_concat.shape

(150, 3)

In [67]:
final_concat.to_csv('./df_max_pool.csv', index=False)

In [68]:
final_concat.shape

(150, 3)

In [74]:
final_concat

,Image,Paragraph,Model
0,https://cs.stanford.edu/people/rak248/VG_100K/...,a plane is on a runway. the sky is very cloudy...,VISUAL
1,https://cs.stanford.edu/people/rak248/VG_100K/...,a man is standing on a beach. he is wearing a ...,VISUAL
2,https://cs.stanford.edu/people/rak248/VG_100K_...,a man is skiing on a hill. the man is wearing ...,VISUAL
3,https://cs.stanford.edu/people/rak248/VG_100K/...,a plane is on a runway. the airplane is blue a...,VISUAL
4,https://cs.stanford.edu/people/rak248/VG_100K/...,a woman is standing on a tennis court. she is ...,VISUAL
...,...,...,...
45,https://cs.stanford.edu/people/rak248/VG_100K_...,a man is standing on the ground. the man is we...,VIS_LANG
46,https://cs.stanford.edu/people/rak248/VG_100K_...,there are two zebras standing on the ground. t...,VIS_LANG
47,https://cs.stanford.edu/people/rak248/VG_100K_...,there is a giraffe standing in a field. there ...,VIS_LANG
48,https://cs.stanford.edu/people/rak248/VG_100K/...,a woman is walking on the sidewalk. the woman ...,VIS_LANG
